In [26]:
from datetime import datetime
import pandas as pd
import random
import numpy as np
import re
import string
import joblib

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Feature Generation

In [27]:
# database = 
# user = 
# password = 
# host = 
# port = 

In [28]:
CONNECTION = f"postgresql://{user}:{password}@{host}:{port}/{database}"

In [29]:
limit = 1000000

all_data = pd.read_sql(
    f"""
    SELECT * FROM public.feed_data feed
    INNER JOIN public.user_data user_data ON feed.user_id = user_data.user_id 
    WHERE feed.action != 'like'
    LIMIT {limit};
    """,
    con=CONNECTION
)

post_data = pd.read_sql(
    """
    SELECT * FROM public.post_text_df
    """,
    con=CONNECTION
)

user_data = pd.read_sql(
    """
    SELECT * FROM public.user_data
    """,
    con=CONNECTION
)

In [30]:
all_data.head()

,timestamp,user_id,post_id,action,target,user_id,gender,age,country,city,exp_group,os,source
0,2021-12-27 13:10:11,122917,1352,view,0,122917,0,31,Russia,Saint Petersburg,1,Android,organic
1,2021-12-27 13:10:30,122917,1250,view,0,122917,0,31,Russia,Saint Petersburg,1,Android,organic
2,2021-12-27 13:13:01,122917,1097,view,0,122917,0,31,Russia,Saint Petersburg,1,Android,organic
3,2021-12-27 13:15:29,122917,1325,view,0,122917,0,31,Russia,Saint Petersburg,1,Android,organic
4,2021-12-27 13:17:17,122917,1150,view,0,122917,0,31,Russia,Saint Petersburg,1,Android,organic


In [31]:
post_data.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [32]:
user_data.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [33]:
wnl = WordNetLemmatizer()

def text_processing(text, wnl=wnl):
    text = text.lower()
    text = ' '.join([wnl.lemmatize(x) for x in text.split(' ')])
    text = text.replace('\n', ' ')
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r"\s{2,}", ' ', text)
    
    return text

def clustering(post_data, tfidf_data):
    len_ = post_data.topic.nunique()
    pca = PCA(n_components=20)
    pca_decomp = pca.fit_transform(tfidf_data - tfidf_data.mean())

    kmeans = KMeans(n_clusters=len_, random_state=42).fit(pca_decomp)
    names = [f'cluster_{i}' for i in range(len_)]

    clusters = pd.DataFrame(
        data = kmeans.transform(pca_decomp),
        columns = names
    )
    
    return clusters

def post_processing(post_data):
    post_data.text = post_data.text.apply(text_processing)
    post_data['text_len'] = post_data.text.apply(lambda x: len(x))
    
    tfidf = TfidfVectorizer()
    tfidf_data = tfidf.fit_transform(post_data.text).toarray()
    
    clusters = clustering(post_data, tfidf_data)
    post_data = pd.concat((post_data, clusters), axis=1)
    post_data = post_data.drop('text', axis=1)
    
    return post_data

In [34]:
post_data = post_processing(post_data)
# post_data.to_sql('my_post_data', con=CONNECTION, index=False, if_exists='replace')
        
post_data.head()

,post_id,topic,text_len,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6
0,1,business,1852,0.392373,0.456890,0.284432,0.446604,0.370241,0.504584,0.426737
1,2,business,2525,0.320809,0.416210,0.205163,0.397738,0.281897,0.477421,0.229817
2,3,business,3122,0.303629,0.379574,0.186743,0.364271,0.295078,0.440719,0.383049
3,4,business,969,0.300977,0.274826,0.151169,0.332349,0.267753,0.332618,0.366012
4,5,business,837,0.276599,0.156344,0.177079,0.286224,0.214361,0.240792,0.351943


In [35]:
def data_proccessing(all_data, categorical):
    all_encoding = pd.DataFrame()
    
    for col in categorical:
        if all_data[col].nunique() < 5:
            result = pd.get_dummies(all_data[col], drop_first=True, prefix=col)
            all_data = pd.concat((all_data, result), axis=1)
            all_data = all_data.drop(col, axis=1)
        else:
            group = all_data[[col, 'target']].groupby([col]).agg('mean').to_dict()['target']
            
            encoding = pd.DataFrame(group.values(), group.keys(), columns=['value'])
            encoding['column'] = col
            all_encoding = pd.concat((all_encoding, encoding), axis=0)
            
            scale = np.mean(list(group.values())) / 10
            all_data[col] = all_data[col].map(group) + np.random.normal(size=len(all_data[col]), scale=scale)
            
#     all_encoding.to_sql('all_encoding', con=CONNECTION, index=True, if_exists='replace')        
    
    return all_data

In [36]:
all_data['month'] = all_data['timestamp'].dt.month
all_data['weekday'] = all_data['timestamp'].dt.weekday
all_data['hour'] = all_data['timestamp'].dt.hour
all_data['minute'] = all_data['timestamp'].dt.minute

time_data = all_data[['month', 'weekday', 'hour', 'minute']]
# time_data.to_sql('my_time_data', con=CONNECTION, index=True, if_exists='replace')        

user_id = all_data.user_id.iloc[:, 0]
all_data = all_data.drop(['action', 'timestamp', 'user_id'], axis=1)

In [37]:
all_data = pd.merge(all_data, post_data, on='post_id')
all_data = pd.concat((all_data, user_id), axis=1)
all_data.head()

,post_id,target,gender,age,country,city,exp_group,os,source,month,...,topic,text_len,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,user_id
0,1352,0,0,31,Russia,Saint Petersburg,1,Android,organic,12,...,politics,13217,0.322219,0.490117,0.337876,0.384112,0.33761,0.555934,0.191679,122917
1,1352,0,1,32,Russia,Yekaterinburg,1,iOS,organic,11,...,politics,13217,0.322219,0.490117,0.337876,0.384112,0.33761,0.555934,0.191679,122917
2,1352,0,1,34,Russia,Timashevsk,0,iOS,ads,11,...,politics,13217,0.322219,0.490117,0.337876,0.384112,0.33761,0.555934,0.191679,122917
3,1352,0,1,32,Russia,Yekaterinburg,1,iOS,organic,12,...,politics,13217,0.322219,0.490117,0.337876,0.384112,0.33761,0.555934,0.191679,122917
4,1352,0,1,34,Russia,Timashevsk,0,iOS,ads,12,...,politics,13217,0.322219,0.490117,0.337876,0.384112,0.33761,0.555934,0.191679,122917


In [38]:
categorical = ['country', 'city', 'exp_group', 'os', 'source', 'topic']
all_data = data_proccessing(all_data, categorical)
all_data.head()

,post_id,target,gender,age,country,city,exp_group,month,weekday,hour,...,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,user_id,os_iOS,source_organic
0,1352,0,0,31,0.112342,0.152058,0.137345,12,0,13,...,0.322219,0.490117,0.337876,0.384112,0.33761,0.555934,0.191679,122917,0,1
1,1352,0,1,32,0.106837,0.065467,0.134298,11,4,11,...,0.322219,0.490117,0.337876,0.384112,0.33761,0.555934,0.191679,122917,1,1
2,1352,0,1,34,0.093473,0.064234,0.094919,11,4,8,...,0.322219,0.490117,0.337876,0.384112,0.33761,0.555934,0.191679,122917,1,0
3,1352,0,1,32,0.123277,0.073130,0.150192,12,0,7,...,0.322219,0.490117,0.337876,0.384112,0.33761,0.555934,0.191679,122917,1,1
4,1352,0,1,34,0.112710,0.062412,0.083644,12,2,23,...,0.322219,0.490117,0.337876,0.384112,0.33761,0.555934,0.191679,122917,1,0


In [39]:
all_encoding = pd.read_sql(
    """
    SELECT * FROM public.all_encoding
    """, con=CONNECTION
)

In [40]:
def update_data(all_data, categorical):

    for col in categorical:
        all_data.loc[:, col] = all_data.loc[:, col].map(all_encoding[all_encoding['column'] == col]\
                                .set_index('index').to_dict()['value'])
        value = all_data[col].mode()[0]
        all_data[col] = all_data[col].fillna(value)
        
    return all_data

In [41]:
categorical = ['country', 'city', 'exp_group']
user_data['exp_group'] = user_data['exp_group'].astype(str)
user_data['os_iOS'] = (user_data['os'] == 'iOS').astype(int)
user_data['source_organic'] = (user_data['source'] == 'organic').astype(int)
user_data = user_data.drop(['os', 'source'], axis=1)
user_data = update_data(user_data, categorical)

In [42]:
all_encoding.head()

,index,value,column
0,Azerbaijan,0.084878,country
1,Belarus,0.189608,country
2,Cyprus,0.099432,country
3,Estonia,0.113736,country
4,Finland,0.146704,country


In [43]:
categorical = ['topic']
post_data = update_data(post_data, categorical)

In [44]:
# CENTERING DATA

user_data.iloc[:, 1:] = StandardScaler().fit_transform(user_data.iloc[:, 1:])
post_data.iloc[:, 1:] = StandardScaler().fit_transform(post_data.iloc[:, 1:])

In [45]:
# user_data.to_sql('my_user_data', con=CONNECTION, index=False, if_exists='replace')
# post_data.to_sql('my_post_data', con=CONNECTION, index=False, if_exists='replace')

In [46]:
user_data.head(2)

,user_id,gender,age,country,city,exp_group,os_iOS,source_organic
0,200,0.902104,0.664568,-0.329883,0.698471,-1.077545,-0.734899,-0.777821
1,201,-1.108519,0.957562,-0.329883,0.885821,-1.039159,-0.734899,-0.777821


In [47]:
post_data.head(2)

,post_id,topic,text_len,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6
0,1,-0.218296,0.520822,1.067342,1.397175,-0.332039,1.658233,0.888716,1.172373,0.159510
1,2,-0.218296,1.078160,0.273682,1.042804,-1.344654,1.089208,-0.306907,0.975985,-2.363697


In [48]:
all_data = all_data.drop(['post_id', 'user_id'], axis=1)
all_data.iloc[:, 1:] = StandardScaler().fit_transform(all_data.iloc[:, 1:])

In [49]:
all_data.head()

,target,gender,age,country,city,exp_group,month,weekday,hour,minute,...,text_len,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,os_iOS,source_organic
0,0,-1.096693,0.368002,-0.214250,0.729799,0.727794,1.257761,-1.478571,-0.277171,-1.126372,...,9.433458,0.227433,1.569554,0.334588,0.838838,0.475579,1.473911,-2.465589,-0.693757,1.825579
1,0,0.911832,0.465043,-0.440438,-1.101795,0.615760,0.028470,0.502512,-0.682478,-0.490134,...,9.433458,0.227433,1.569554,0.334588,0.838838,0.475579,1.473911,-2.465589,1.441426,1.825579
2,0,0.911832,0.659126,-0.989452,-1.127889,-0.831935,0.028470,0.502512,-1.290438,0.955861,...,9.433458,0.227433,1.569554,0.334588,0.838838,0.475579,1.473911,-2.465589,1.441426,-0.547772
3,0,0.911832,0.465043,0.234993,-0.939713,1.200072,1.257761,-1.478571,-1.493091,1.129381,...,9.433458,0.227433,1.569554,0.334588,0.838838,0.475579,1.473911,-2.465589,1.441426,1.825579
4,0,0.911832,0.659126,-0.199131,-1.166416,-1.246423,1.257761,-0.488029,1.749362,-1.531251,...,9.433458,0.227433,1.569554,0.334588,0.838838,0.475579,1.473911,-2.465589,1.441426,-0.547772


In [50]:
# Data to train model

all_data.shape

(1000000, 21)